<a href="https://colab.research.google.com/github/LordGum/Telegram_bot/blob/main/Telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip  install pyTelegramBotAPI

In [16]:
import telebot
from telebot import types
import sqlite3

База данных

In [17]:
import sqlite3
import json

class Database:
    def __init__(self, db_path: str = "telegram_bot_project_1.db"):
        self.db_path = db_path
        self.init_db()

    def init_db(self):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS todos (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER NOT NULL,
                    text TEXT NOT NULL,
                    done BOOLEAN DEFAULT FALSE,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            conn.execute('''
                CREATE TABLE IF NOT EXISTS user_stats (
                    user_id INTEGER PRIMARY KEY,
                    first_seen TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    last_seen TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            conn.execute('''
                CREATE TABLE IF NOT EXISTS command_logs (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    user_id INTEGER NOT NULL,
                    command TEXT NOT NULL,
                    executed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            conn.commit()

    def add_todo(self, user_id: int, text: str):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute(
                'INSERT INTO todos (user_id, text) VALUES (?, ?)',
                (user_id, text)
            )
            conn.commit()
            return cursor.lastrowid

    def get_todos(self, user_id: int, offset: int = 0, limit: int = 10):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''
                SELECT id, text, done, created_at
                FROM todos
                WHERE user_id = ?
                ORDER BY created_at DESC
                LIMIT ? OFFSET ?
            ''', (user_id, limit, offset))

            return [{
                'id': row[0],
                'text': row[1],
                'done': bool(row[2]),
                'created_at': row[3]
            } for row in cursor.fetchall()]

    def mark_todo_done(self, user_id: int, todo_id: int):
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''
                UPDATE todos SET done = TRUE
                WHERE id = ? AND user_id = ?
            ''', (todo_id, user_id))
            conn.commit()
            return cursor.rowcount > 0

In [18]:
from google.colab import userdata
token = userdata.get('token')
weather_api = userdata.get('weather_token')

In [19]:
bot = telebot.TeleBot(token)

@bot.message_handler(commands=['start'])
def start(message):
  bot.send_message(message.chat.id, "привет")


**Команды todo**

In [20]:
db = Database()

@bot.message_handler(commands=['todo'])
def todo_command(message):
  parts = message.text.strip().split(maxsplit=2)

  if len(parts) < 2:
    bot.send_message(message.chat.id, "нехватает параметров, введите команду заново")
    return

  command = parts[1].lower()
  user_id = message.from_user.id
  if command == "add" and len(parts) > 2:
    task_text = parts[2]
    task_id = db.add_todo(user_id, task_text)
    bot.send_message(message.chat.id, f"Задача #{task_id} добавлена: {task_text}")

  elif command == "list":
    todos = db.get_todos(user_id)
    if not todos:
      bot.send_message(message.chat.id, "Список задач пуст")
    else:
      response = []
      for todo in todos:
          status = "✅" if todo['done'] else " □ "
          response.append(f"{status} #{todo['id']}: {todo['text']}")

      response_text = "Ваши задачи:\n" + "\n".join(response[::-1])
      bot.send_message(message.chat.id, f"{response_text}")

  elif command == "done" and len(parts) > 2:
    task_id = int(parts[2])
    if db.mark_todo_done(user_id, task_id):
        bot.send_message(message.chat.id, f"✅ Задача #{task_id} выполнена!")
    else:
        bot.send_message(message.chat.id, "❌ Задача не найдена")

  else:
    bot.send_message(message.chat.id, "Ошибка команды /list")

**Команда weather**

In [21]:
import requests

@bot.message_handler(commands=['weather'])
def weather_command(message):
  answer = ''
  text = message.text.split()
  if len(text) <= 0:
    answer = "Ошибка. Неверная команда"
  if len(text) == 1:
    answer = "Введите название города"
    bot.send_message(message.chat.id, f"{answer}")
    bot.register_next_step_handler(message, city_command)
  else:
    answer = f"Узнаю погоду в {' '.join(text[1:])}"
    bot.send_message(message.chat.id, f"{answer}")

def city_command(message):
  try:
    city = message.text
    res = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api}&units=metric')
    data = json.loads(res.text)
    answer = f'''
сейчас в {city}
температура {data['main']['temp']} градусов
    '''
    bot.reply_to(message, answer)
  except Exception as e:
    bot.reply_to(message, "Произошла ошибка")



**Команда help**

In [22]:
@bot.message_handler(commands=['help'])
def help_command(message):
  text = r'''/todo - управление задачами
/weather <город> - погода
/rate <базовая> <валюты> - курсы валют
/fileinfo - информация о файлах
/stats - статистика бота
/help - помощь
  '''
  bot.send_message(message.chat.id, f"{text}")

**Команда rate**

In [23]:
!pip install CurrencyConverter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.4/633.4 kB 8.4 MB/s eta 0:00:00


In [24]:
from currency_converter import CurrencyConverter
from telebot import types

converter = CurrencyConverter()

@bot.message_handler(commands=['rate'])
def rate_command(message):
  bot.send_message(message.chat.id, "Введите сумму для конвертации")
  bot.register_next_step_handler(message, convert)

def convert(message):
  global amount
  try:
    amount = int(message.text.strip())
  except ValueError:
    bot.send_message(message.chat.id, 'Неверный формат. Впишите сумму')
    bot.register_next_step_handler(message, convert)
    return
  if amount > 0:
    markup = types.InlineKeyboardMarkup(row_width=2)
    btn1 = types.InlineKeyboardButton('USD/EUR', callback_data='usd/eur')
    btn2 = types.InlineKeyboardButton('EUR/USD', callback_data='eur/usd')
    btn3 = types.InlineKeyboardButton('USD/GBP', callback_data='usd/gbp')
    markup.add(btn1, btn2, btn3)
    bot.send_message(message.chat.id, 'Выберите пару валют', reply_markup=markup)
  else:
    bot.send_message(message.chat.id, 'Число должно быть больше 0. Впишите сумму')
    bot.register_next_step_handler(message, convert)


@bot.callback_query_handler(func=lambda call: True)
def callback(call):
    values = call.data.upper().split('/')
    res = converter.convert(amount, values[0], values[1])
    bot.send_message(call.message.chat.id, f'Получается: {amount} {values[0]} - это {round(res, 2)} {values[1]}')


# **Команда fileinfo**

In [25]:
import hashlib
import os

class FileService:
    @staticmethod
    def get_file_info(file_path: str):
        file_stats = os.stat(file_path)

        sha256_hash = hashlib.sha256()
        f = open(file_path)
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

        return {
            'filename': os.path.basename(file_path),
            'size_kb': round(file_stats.st_size / 1024, 2),
            'sha256': sha256_hash.hexdigest()
        }

In [26]:
import tempfile
import os

file_service = FileService()

@bot.message_handler(commands=['fileinfo'])
def fileinfo_command(message):
  bot.send_message(message.chat.id, " Отправьте мне любой файл, и я покажу информацию о нём")
  bot.register_next_step_handler(message, handle_file)


@bot.message_handler(content_types=['document', 'photo', 'video', 'audio', 'voice'])
def handle_file(message):
    try:
        if message.document:
            file_info = bot.get_file(message.document.file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            file_name = message.document.file_name
            file_size = message.document.file_size

        elif message.photo:
            file_info = bot.get_file(message.photo[-1].file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            file_name = f"photo_{file_info.file_id}.jpg"
            file_size = message.photo[-1].file_size

        elif message.audio:
            file_info = bot.get_file(message.audio.file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            file_name = message.audio.file_name or f"audio_{file_info.file_id}.mp3"
            file_size = message.audio.file_size

        elif message.video:
            file_info = bot.get_file(message.video.file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            file_name = message.video.file_name or f"video_{file_info.file_id}.mp4"
            file_size = message.video.file_size

        elif message.voice:
            file_info = bot.get_file(message.voice.file_id)
            downloaded_file = bot.download_file(file_info.file_path)
            file_name = f"voice_{file_info.file_id}.ogg"
            file_size = message.voice.file_size
        else:
          bot.send_message(message.chat.id, "Кажется, я не знаю такой тип файлов.")

        sha256_hash = hashlib.sha256(downloaded_file).hexdigest()

        info_message = (
            f"Информация о файле:\n\n"
            f"Имя: {file_name}\n\n"
            f"Размер: {file_size}\n\n"
            f"SHA-256: {sha256_hash}\n\n"
        )

        bot.send_message(message.chat.id, info_message)

    except Exception as e:
        bot.send_message(message.chat.id, f"Произошла ошибка при обработке файла: {str(e)}")

In [ ]:
@bot.message_handler()
def handle_other_messages(message):
  if message.text.startswith('/'):
      bot.send_message(message.chat.id, "Ошибка. Неизвестная команда")
  else:
    bot.send_message(message.chat.id, "неизвестная команда, если нужна помощь напишите /help")

bot.polling(none_stop=True)